In [8]:
LANGHCHAIN_TRACINB_V2 = "true"
LANGCHAIN_API_KEY = "ls__ef7c69da49cf48719e4d7c4b33a3debd"
import os
os.environ['LANGCHAIN_API_KEY'] = LANGCHAIN_API_KEY

In [9]:
import json

with open("train.json", "r") as file:
    dataset = json.load(file)
file.close()

In [10]:
from langchain_community.llms import Ollama

llm = Ollama(model='openchat')

In [4]:
llm.invoke("hello")

' Hello! How can I assist you today? If you have any questions or need information, feel free to ask.'

In [11]:
import joblib

top_preds = joblib.load('./vecs/top_preds')
true_labels = joblib.load('./vecs/true_labels')
doc_labels = joblib.load('./vecs/doc_label')

In [12]:
len(top_preds), len(true_labels), len(dataset[0]['annotations'][0]['result'])

(91, 91, 91)

In [13]:
for p in top_preds:
    if len(p)!=5:
        print(len(p))

In [14]:
curr_sent = dataset[0]['annotations'][0]['result'][0]['value']['text']
curr_sent

'      IN THE HIGH COURT OF KARNATAKA,\n          CIRCUIT BENCH AT GULBARGA\n\nDATED THIS THE 22ND DAY OF FEBRUARY, 2013'

In [9]:
prompt = f'''You are a specialized system focused on sentence classification of court opinion.
        RHETORICAL ROLE:
        Rhetorical roles in legal writing refer to the distinct functions or purposes that different parts of a document, such as
        a legal opinion, serve in conveying information, persuading the reader, and constructing a coherent argument. These
        roles encompass various elements like factual background, legal principles, arguments, counter arguments, and
        conclusions, each contributing to the document's overall persuasive and informative structure.
        Your task is to label each sentence in the document with one of the following predefined rhetorical roles: ['Preamble', 'Facts',
        'Ruling by Lower Court', 'Issues', 'Argument by Petitioner', 'Argument by Respondent', 'Analysis', 'Statute',
        'Precedent Relied', 'Precedent Not Relied', 'Ratio of the decision', 'Ruling by Present Court', 'NONE']
        The definition of each rhetorical role is given below:
        Preamble: A typical judgement would start with the court name, the details of parties, lawyers and judges' names, Headnotes. This section typically would end with a keyword like (JUDGEMENT or ORDER etc.) Some supreme court cases also have HEADNOTES, ACTS section. They are also part of Preamble.
        Facts: This refers to the chronology of events (but not judgement by lower court) that led to filing the case, and how the case evolved over time in the legal system (e.g., First Information Report at a police station, filing an appeal to the Magistrate, etc.) Depositions and proceedings of current court. Summary of lower court proceedings
        Ruling by Lower Court: Judgments given by the lower courts (Trial Court, High Court) based on which the present appeal was made (to the Supreme Court or high court). The verdict of the lower Court, Analysis & the ratio behind the judgement by the lower Court is annotated with this label.
        Issues: Some judgements mention the key points on which the verdict needs to be delivered. Such Legal Questions Framed by the Court are ISSUES. E.g. “he point emerge for determination is as follow:- (i) Whether on 06.08.2017 the accused persons in furtherance of their common intention intentionally caused the death of the deceased by assaulting him by means of axe ?”
        Argument by Petitioner: Arguments by petitioners' lawyers. Precedent cases argued by petitioner lawyers fall under this but when court discusses them later then they belong to either the relied / not relied upon category. E.g. “learned counsel for petitioner argued that …”
        Argument by Respondent: Arguments by respondents lawyers. Precedent cases argued by respondent lawyers fall under this but when court discusses them later then they belong to either the relied / not relied upon category. E.g. “learned counsel for the respondent argued that …”
        Analysis: Courts discussion on the evidence,facts presented,prior cases and statutes. These are views of the court. Discussions on how the law is applicable or not applicable to current case. Observations(non binding) from court. It is the parent tag for 3 tags: PRE_RLEIED, PRE_NOT_RELIED and STATUTE i.e. Every statement which belong to these 3 tags should also be marked as ANALYSIS. E.g. “Post Mortem Report establishes that .. “ E.g. “In view of the abovementioned findings, it is evident that the ingredients of Section 307 have been made out ….”
        Statute : Text in which the court discusses Established laws, which can come from a mixture of sources - Acts , Sections, Articles, Rules, Order, Notices, Notifications, Quotations directly from the bare act, and so on. Statute will have both the tags Analysis + Statute. E.g. “Court had referred to Section 4 of the Code, which reads as under: "4. Trial of offences under the Indian Penal Code and other laws.-- (1) All offences under the Indian Penal Code (45 of 1860) shall be investigated, inquired into, tried, and otherwise dealt with according to the provisions hereinafter contained”
        Precedent Relied: Sentences in which the court discusses prior case documents, discussions and decisions which were relied upon by the court for final decisions. So Precedent will have both the tags Analysis + Precedent. E.g. This Court in Jage Ram v. State of Haryana3 held that: "12. For the purpose of conviction under Section 307 IPC, ….. “
        Precedent Not Relied: Sentences in which the court discusses prior case documents, discussions and decisions which were not relied upon by the court for final decisions. It could be due to the fact that the situation in that case is not relevant to the current case.. E.g. This Court in Jage Ram v. State of Haryana3 held that: "12. For the purpose of conviction under Section 307 IPC, ….. “
        Ratio of the decision: Main Reason given for the application of any legal principle to the legal issue. This is the result of the analysis by the court. This typically appears right before the final decision. This is not the same as “Ratio Decidendi” taught in the Legal Academic curriculum. E.g. “The finding that the sister concern is eligible for more deduction under Section 80HHC of the Act is based on mere surmise and conjectures also does not arise for consideration.”
        Ruling by Present Court: Final decision + conclusion + order of the Court following from the natural / logical outcome of the rationale. E.g. “In the result, we do not find any merit in this appeal. The same fails and is hereby dismissed.”
        NONE: 	If a sentence does not belong to any of the above categories. E.g. “We have considered the submissions made by learned counsel for the parties and have perused the record.”
        
        You are given top 5 most similar sentences and their corresponding labels:

        '''

for p in top_preds[0]:
    prompt+= f"SENTENCE: {p[0].strip()}" + f"\nLABEL: {p[1]}\n"

pred_prompt = f'''Your task is to predict the label of the current sentence using the previous provided most similar sentences.
Your output should only be the label of the current sentence as a string and nothing else.
Now do prediction for this current sentence: {curr_sent.strip()}'''

prompt += pred_prompt
        

In [10]:
print(prompt)

You are a specialized system focused on sentence classification of court opinion.
        RHETORICAL ROLE:
        Rhetorical roles in legal writing refer to the distinct functions or purposes that different parts of a document, such as
        a legal opinion, serve in conveying information, persuading the reader, and constructing a coherent argument. These
        roles encompass various elements like factual background, legal principles, arguments, counter arguments, and
        conclusions, each contributing to the document's overall persuasive and informative structure.
        Your task is to label each sentence in the document with one of the following predefined rhetorical roles: ['Preamble', 'Facts',
        'Ruling by Lower Court', 'Issues', 'Argument by Petitioner', 'Argument by Respondent', 'Analysis', 'Statute',
        'Precedent Relied', 'Precedent Not Relied', 'Ratio of the decision', 'Ruling by Present Court', 'NONE']
        The definition of each rhetorical role is g

In [11]:
response = llm.invoke(prompt).strip()

In [12]:
print(response)

PREAMBLE


In [15]:
from tqdm import tqdm

In [16]:
labels = ['Preamble', 'Facts', 'Ruling by Lower Court', 'Issues', 'Argument by Petitioner', 'Argument by Respondent', 'Analysis', 'Statute', 'Precedent Relied', 'Precedent Not Relied', 'Ratio of the decision', 'Ruling by Present Court', 'NONE']

In [60]:
def run_preds(data):
    preds = []
    for i in tqdm(range(3, len(data))):
        curr_sent = data[i]['value']['text']

        prompt = f'''Your task is to perform classification of a given sentence derived from a court opinion. The classification serves the purpose of rhetorical roles assignment.

Instructions:
1. You need to classify the given sentence by associating it to exactly one rhetorical role.
2. For reference, you will be provided with a sample set of semantically similar sentences along with their corresponding rhetorical roles.
3. Choose rhetorical roles from provided sample or based on available information as preferred.

Below we formalize rhetorical roles.
RHETORICAL ROLES: # Try w/ and w/o lowercasing this or we can discard this too.
Rhetorical roles in legal writing explain the varied purposes served by different document components, like a legal opinion, in conveying information, persuading readers, and crafting a coherent argument. These roles encompass various elements like factual background, legal principles, arguments, counter arguments, and conclusions, each contributing to the document's overall persuasive and informative structure.

Following is the list of predefined rhetorical roles:
['Preamble', 'Facts', 'Ruling by Lower Court', 'Issues', 'Argument by Petitioner', 'Argument by Respondent', 'Analysis', 'Statute', 'Precedent Relied', 'Precedent Not Relied', 'Ratio of the decision', 'Ruling by Present Court', 'NONE']

The definition of each of the stated rhetorical role is given below:
Preamble: A typical judgement would start with the court name, the details of parties, lawyers and judges' names, Headnotes. This section typically would end with a keyword like (JUDGEMENT or ORDER etc.) Some supreme court cases also have HEADNOTES, ACTS section. They are also part of Preamble.
Facts: 	This refers to the chronology of events (but not judgement by lower court) that led to filing the case, and how the case evolved over time in the legal system (e.g., First Information Report at a police station, filing an appeal to the Magistrate, etc.). Depositions and proceedings of current court. Summary of lower court proceedings.
Ruling by Lower Court: Judgments given by the lower courts (Trial Court, High Court) based on which the present appeal was made (to the Supreme Court or high court). The verdict of the lower Court, Analysis & the ratio behind the judgement by the lower Court is annotated with this label.
Issues: Some judgements mention the key points on which the verdict needs to be delivered. Such Legal Questions Framed by the Court are ISSUES.
Argument by Petitioner: Arguments by petitioners' lawyers. Precedent cases argued by petitioner lawyers fall under this but when court discusses them later then they belong to either the relied / not relied upon category.
Argument by Respondent: Arguments by respondents lawyers. Precedent cases argued by respondent lawyers fall under this but when court discusses them later then they belong to either the relied / not relied upon category.
Analysis: Courts discussion on the evidence,facts presented,prior cases and statutes. These are views of the court. Discussions on how the law is applicable or not applicable to current case. Observations(non binding) from court. It is the parent tag for 3 tags: PRE_RLEIED, PRE_NOT_RELIED and STATUTE i.e. Every statement which belong to these 3 tags should also be marked as ANALYSIS
Statute : Text in which the court discusses Established laws, which can come from a mixture of sources - Acts , Sections, Articles, Rules, Order, Notices, Notifications, Quotations directly from the bare act, and so on. Statute will have both the tags Analysis + Statute
Precedent Relied: Sentences in which the court discusses prior case documents, discussions and decisions which were relied upon by the court for final decisions. So Precedent will have both the tags Analysis + Precedent
Precedent Not Relied: Sentences in which the court discusses prior case documents, discussions and decisions which were not relied upon by the court for final decisions. It could be due to the fact that the situation in that case is not relevant to the current case.
Ratio of the decision: Main Reason given for the application of any legal principle to the legal issue. This is the result of the analysis by the court. This typically appears right before the final decision. This is not the same as “Ratio Decidendi” taught in the Legal Academic curriculum.
Ruling by Present Court: Final decision + conclusion + order of the Court following from the natural / logical outcome of the rationale
NONE: If a sentence does not belong to any of the above categories.

'''    

        trg_sent = ". ".join([item.strip() for item in curr_sent.strip().split("\n")])
        prompt+=f"Target sentence:\n{trg_sent}\n\n"

        for p in top_preds[i]:
            sent = ". ".join([item.strip() for item in p[0].strip().split("\n")])
                    
            prompt+= f"sample_sentence: {sent}" + f"\nassigned_role: {p[1][0] + p[1][1:].lower()}\n"

        pred_prompt = f'''Do not generate anything except the rhetorical role for the given sentence.'''

        prompt += pred_prompt
        if i==10:
            print(prompt)
            break

In [61]:
run_preds(dataset[0]['annotations'][0]['result'])

  8%|▊         | 7/88 [00:00<00:00, 38329.15it/s]

Your task is to perform classification of a given sentence derived from a court opinion. The classification serves the purpose of rhetorical roles assignment.

Instructions:
1. You need to classify the given sentence by associating it to exactly one rhetorical role.
2. For reference, you will be provided with a sample set of semantically similar sentences along with their corresponding rhetorical roles.
3. Choose rhetorical roles from provided sample or based on available information as preferred.

Below we formalize rhetorical roles.
RHETORICAL ROLES: # Try w/ and w/o lowercasing this or we can discard this too.
Rhetorical roles in legal writing explain the varied purposes served by different document components, like a legal opinion, in conveying information, persuading readers, and crafting a coherent argument. These roles encompass various elements like factual background, legal principles, arguments, counter arguments, and conclusions, each contributing to the document's overall p

In [40]:
def run_preds(data):
    preds = []
    for i in tqdm(range(len(data))):
        curr_sent = data[i]['value']['text']

        prompt = f'''Your task is to perform classification of a given sentence derived from a court opinion. The classification serves the purpose of rhetorical roles assignment.

Instructions:
1. You need to classify the given sentence by associating it to exactly one rhetorical role.
2. For reference, you will be provided with a sample set of semantically similar sentences along with their corresponding rhetorical roles.
3. Choose rhetorical roles from provided sample or based on available information as preferred.

Below we formalize rhetorical roles.
RHETORICAL ROLES: # Try w/ and w/o lowercasing this or we can discard this too.
Rhetorical roles in legal writing explain the varied purposes served by different document components, like a legal opinion, in conveying information, persuading readers, and crafting a coherent argument. These roles encompass various elements like factual background, legal principles, arguments, counter arguments, and conclusions, each contributing to the document's overall persuasive and informative structure.

Following is the list of predefined rhetorical roles:
['Preamble', 'Facts', 'Ruling by Lower Court', 'Issues', 'Argument by Petitioner', 'Argument by Respondent', 'Analysis', 'Statute', 'Precedent Relied', 'Precedent Not Relied', 'Ratio of the decision', 'Ruling by Present Court', 'NONE']

The definition of each of the stated rhetorical role is given below:
Preamble: A typical judgement would start with the court name, the details of parties, lawyers and judges' names, Headnotes. This section typically would end with a keyword like (JUDGEMENT or ORDER etc.) Some supreme court cases also have HEADNOTES, ACTS section. They are also part of Preamble.
Facts: 	This refers to the chronology of events (but not judgement by lower court) that led to filing the case, and how the case evolved over time in the legal system (e.g., First Information Report at a police station, filing an appeal to the Magistrate, etc.). Depositions and proceedings of current court. Summary of lower court proceedings.
Ruling by Lower Court: Judgments given by the lower courts (Trial Court, High Court) based on which the present appeal was made (to the Supreme Court or high court). The verdict of the lower Court, Analysis & the ratio behind the judgement by the lower Court is annotated with this label.
Issues: Some judgements mention the key points on which the verdict needs to be delivered. Such Legal Questions Framed by the Court are ISSUES.
Argument by Petitioner: Arguments by petitioners' lawyers. Precedent cases argued by petitioner lawyers fall under this but when court discusses them later then they belong to either the relied / not relied upon category.
Argument by Respondent: Arguments by respondents lawyers. Precedent cases argued by respondent lawyers fall under this but when court discusses them later then they belong to either the relied / not relied upon category.
Analysis: Courts discussion on the evidence,facts presented,prior cases and statutes. These are views of the court. Discussions on how the law is applicable or not applicable to current case. Observations(non binding) from court. It is the parent tag for 3 tags: PRE_RLEIED, PRE_NOT_RELIED and STATUTE i.e. Every statement which belong to these 3 tags should also be marked as ANALYSIS
Statute : Text in which the court discusses Established laws, which can come from a mixture of sources - Acts , Sections, Articles, Rules, Order, Notices, Notifications, Quotations directly from the bare act, and so on. Statute will have both the tags Analysis + Statute
Precedent Relied: Sentences in which the court discusses prior case documents, discussions and decisions which were relied upon by the court for final decisions. So Precedent will have both the tags Analysis + Precedent
Precedent Not Relied: Sentences in which the court discusses prior case documents, discussions and decisions which were not relied upon by the court for final decisions. It could be due to the fact that the situation in that case is not relevant to the current case.
Ratio of the decision: Main Reason given for the application of any legal principle to the legal issue. This is the result of the analysis by the court. This typically appears right before the final decision. This is not the same as “Ratio Decidendi” taught in the Legal Academic curriculum.
Ruling by Present Court: Final decision + conclusion + order of the Court following from the natural / logical outcome of the rationale
NONE: If a sentence does not belong to any of the above categories.

'''    

        trg_sent = ". ".join([item.strip() for item in curr_sent.strip().split("\n")])
        prompt+=f"Target sentence:\n{trg_sent}\n\n"

        for p in top_preds[i]:
            sent = ". ".join([item.strip() for item in p[0].strip().split("\n")])
                    
            prompt+= f"sample_sentence: {sent}" + f"\nassigned_role: {p[1][0] + p[1][1:].lower()}\n"

        pred_prompt = f'''Do not generate anything except the rhetorical role for the given sentence.'''

        prompt += pred_prompt
    
        response = llm.invoke(prompt).strip()
        preds.append(response)
    
    return preds

In [41]:
llm_labels = run_preds(dataset[0]['annotations'][0]['result'])
len(llm_labels)

100%|██████████| 91/91 [00:41<00:00,  2.21it/s]


91

In [47]:
llm_labels[-10:], true_labels[-10:]

(['Argument by Petitioner',
  'Facts',
  'Issues',
  'Issues',
  'Rpc',
  'Rpc',
  'Ruling by Present Court',
  'Ruling by Present Court',
  'Ruling by Present Court',
  'NONE'],
 ['ANALYSIS',
  'ANALYSIS',
  'RATIO',
  'ANALYSIS',
  'RPC',
  'RPC',
  'RPC',
  'RPC',
  'RPC',
  'NONE'])

In [52]:
label_abbrv = {
    'preamble': 'PREAMBLE',
    "'preamble'": 'PREAMBLE',
    'facts': 'FAC',
    'fac': 'FAC',
    'ruling by lower court': 'RLC',
    'rlc': 'RLC',
    'issues': 'ISSUE',
    'issue': 'ISSUE',
    'argument by petitioner': 'ARG_PETITIONER',
    'argument_by_petitioner': 'ARG_PETITIONER',
    'arg_petitioner': 'ARG_PETITIONER',
    'argument by respondent': 'ARG_RESPONDENT',
    'arg_respondent': 'ARG_RESPONDENT',
    'analysis': 'ANALYSIS',
    'pre_analysis': 'ANALYSIS',
    'statute': 'STA',
    'precedent relied': 'PRE_RELIED',
    'pre_relied': 'PRE_RELIED',
    'precedent not relied': 'PRE_NOT_RELIED',
    'pre_not_relied': 'PRE_NOT_RELIED',
    'ratio': 'Ratio',
    'ratio of the decision': 'Ratio',
    'ratio of decision': 'Ratio',
    'ratio_of_decision': 'Ratio',
    'ruling by present court': 'RPC',
    'ruling_by_present_court': 'RPC',
    'rpc': 'RPC',
    'none': 'NONE'
}
  
abbrv_label = {v: k for k, v in label_abbrv.items()}
abbrv_label

{'PREAMBLE': "'preamble'",
 'FAC': 'fac',
 'RLC': 'rlc',
 'ISSUE': 'issue',
 'ARG_PETITIONER': 'arg_petitioner',
 'ARG_RESPONDENT': 'arg_respondent',
 'ANALYSIS': 'pre_analysis',
 'STA': 'statute',
 'PRE_RELIED': 'pre_relied',
 'PRE_NOT_RELIED': 'pre_not_relied',
 'Ratio': 'ratio_of_decision',
 'RPC': 'rpc',
 'NONE': 'none'}

In [53]:
preds = []
trues = []
for i, label in enumerate(llm_labels):
    try:
        preds.append(label_abbrv[label.lower()])
        trues.append(true_labels[i])
    except:
        # preds.append("none")
        # trues.append(true_labels[i].lower())
        print(i, label.lower())
        print(true_labels[i].lower())

1 assigned_role: preamble
preamble
31 argument by prosecution
arg_petitioner


In [54]:
len(preds), len(trues)

(89, 89)

In [55]:
preds[:10], trues[:10]

(['NONE',
  'ISSUE',
  'PREAMBLE',
  'ANALYSIS',
  'ISSUE',
  'FAC',
  'FAC',
  'FAC',
  'FAC',
  'FAC'],
 ['PREAMBLE',
  'PREAMBLE',
  'PREAMBLE',
  'NONE',
  'FAC',
  'FAC',
  'FAC',
  'FAC',
  'FAC',
  'FAC'])

In [56]:
from sklearn.metrics import classification_report, accuracy_score, f1_score
acc = accuracy_score(trues, preds)
f1 = f1_score(trues, preds, average="macro")

In [57]:
acc, f1

(0.30337078651685395, 0.28017830123093285)

In [59]:
print(classification_report(trues, preds))

                precision    recall  f1-score   support

      ANALYSIS       0.50      0.22      0.31         9
ARG_PETITIONER       0.18      0.30      0.22        10
ARG_RESPONDENT       0.24      0.29      0.26        17
           FAC       0.42      0.83      0.56        12
         ISSUE       0.00      0.00      0.00         0
          NONE       0.25      0.50      0.33         2
      PREAMBLE       0.50      0.33      0.40         3
    PRE_RELIED       0.00      0.00      0.00        26
         RATIO       0.00      0.00      0.00         1
           RLC       0.00      0.00      0.00         4
           RPC       1.00      1.00      1.00         5

      accuracy                           0.30        89
     macro avg       0.28      0.32      0.28        89
  weighted avg       0.25      0.30      0.26        89



/home/silly_ronny/miniconda3/envs/langchain/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/silly_ronny/miniconda3/envs/langchain/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/silly_ronny/miniconda3/envs/langchain/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, 